In [14]:
import tensorflow as tf
from tensorflow.keras import utils, models, layers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from gui import create_model_gui

In [ ]:
model, params_dict = create_model_gui()
params_dict

In [2]:
from datetime import date
#date time objects for each nba season

nba_seasons = {
    '2000-2001': {'start': date(2000, 10, 31), 'end': date(2001, 4, 18)},
    '2001-2002': {'start': date(2001, 10, 30), 'end': date(2002, 4, 17)},
    '2002-2003': {'start': date(2002, 10, 29), 'end': date(2003, 4, 16)},
    '2003-2004': {'start': date(2003, 10, 28), 'end': date(4, 4, 14)},
    '2004-2005': {'start': date(2004, 11, 2), 'end': date(2005, 4, 20)},
    '2005-2006': {'start': date(2005, 11, 1), 'end': date(2006, 4, 19)},
    '2006-2007': {'start': date(2006, 10, 31), 'end': date(2007, 4, 18)},
    '2007-2008': {'start': date(2007, 10, 30), 'end': date(2008, 4, 16)},
    '2008-2009': {'start': date(2008, 10, 28), 'end': date(2009, 4, 15)},
    '2009-2010': {'start': date(2009, 10, 27), 'end': date(2010, 4, 14)},
    '2010-2011': {'start': date(2010, 10, 26), 'end': date(2011, 4, 13)},
    '2011-2012': {'start': date(2011, 12, 25), 'end': date(2012, 4, 26)},
    '2012-2013': {'start': date(2012, 10, 30), 'end': date(2013, 4, 17)},
    '2013-2014': {'start': date(2013, 10, 29), 'end': date(2014, 4, 16)},
    '2014-2015': {'start': date(2014, 10, 28), 'end': date(2015, 4, 15)},
    '2015-2016': {'start': date(2015, 10, 27), 'end': date(2016, 4, 13)},
    '2016-2017': {'start': date(2016, 10, 25), 'end': date(2017, 4, 12)},
    '2017-2018': {'start': date(2017, 10, 17), 'end': date(2018, 4, 11)},
    '2018-2019': {'start': date(2018, 10, 16), 'end': date(2019, 4, 10)},
    '2019-2020': {'start': date(2019, 10, 22), 'end': date(2020, 9, 28)},  # Adjusted end date
    '2020-2021': {'start': date(2020, 12, 22), 'end': date(2021, 7, 22)},  # Adjusted end date
    '2021-2022': {'start': date(2021, 10, 19), 'end': date(2022, 4, 10)},
    '2022-2023': {'start': date(2022, 10, 18), 'end': date(2023, 4, 9)}
}


Our first step is to create our training and testing data from all the data we downloaded.

In [3]:
from game_data_ingest import *

some_dates = date_range(nba_seasons[params_dict['Season']]['start'], nba_seasons[params_dict['Season']]['end'])
put_dates_in_db(some_dates)

with sqlite3.connect("games.db") as conn:
    c = conn.cursor()
    q = f"SELECT COUNT(*) FROM games;"
    c.execute(q)
    print("Total count of games in the database:", c.fetchone()[0])

with sqlite3.connect("games.db") as conn:
    df = generate_training_data_for_season(
        conn, nba_seasons[params_dict['Season']]['start'], nba_seasons[params_dict['Season']]['end']
    )
    print("---- Training Data ----")
    print(df)

142 dates have to be scraped
Starting crawl
Finished crawl
Total count of games in the database: 1574
---- Training Data ----
      hometeam_fieldGoalsMade  hometeam_fieldGoalsAttempted  \
0                   35.524590                     78.786885   
1                   37.642857                     82.875000   
2                   38.338983                     81.864407   
3                   36.482759                     77.810345   
4                   34.983607                     79.229508   
...                       ...                           ...   
1168                34.844156                     80.909091   
1169                33.227848                     78.455696   
1170                33.833333                     79.166667   
1171                34.233766                     78.792208   
1172                35.571429                     83.597403   

      hometeam_threePointersMade  hometeam_threePointersAttempted  \
0                       6.098361                

In [7]:
df.head()

,hometeam_fieldGoalsMade,hometeam_fieldGoalsAttempted,hometeam_threePointersMade,hometeam_threePointersAttempted,hometeam_freeThrowsMade,hometeam_freeThrowsAttempted,hometeam_reboundsOffensive,hometeam_reboundsDefensive,hometeam_assists,hometeam_steals,...,awayteam_freeThrowsMade,awayteam_freeThrowsAttempted,awayteam_reboundsOffensive,awayteam_reboundsDefensive,awayteam_assists,awayteam_steals,awayteam_blocks,awayteam_foulsPersonal,awayteam_points,winner_is_home_team
0,35.524590,78.786885,6.098361,17.016393,19.196721,25.311475,11.327869,30.426230,19.409836,7.360656,...,17.126984,24.936508,12.222222,31.142857,18.873016,6.063492,6.301587,21.952381,91.555556,1
1,37.642857,82.875000,6.910714,17.750000,18.089286,23.178571,11.964286,30.267857,22.125000,8.178571,...,19.103448,25.189655,11.293103,30.431034,19.379310,7.310345,4.689655,20.913793,96.034483,0
2,38.338983,81.864407,5.305085,15.661017,19.033898,28.305085,13.271186,31.491525,22.881356,6.508475,...,18.836066,25.081967,13.868852,30.983607,24.032787,7.098361,5.754098,21.754098,97.622951,1
3,36.482759,77.810345,4.465517,12.586207,17.965517,23.310345,11.620690,30.362069,23.810345,8.551724,...,17.150000,25.083333,12.383333,31.316667,19.083333,6.033333,6.350000,22.266667,91.666667,1
4,34.983607,79.229508,4.344262,13.114754,17.180328,25.098361,12.262295,31.262295,19.049180,6.016393,...,18.767857,24.517857,12.232143,31.964286,22.875000,9.785714,5.678571,19.535714,101.107143,1


We have 27 - 1 = 26 input parameters. The remaining value is what we are trying to predict: whether the home team won.

In [8]:
#make arrays for X and Y data
X = df.drop('winner_is_home_team', axis=1).values
y = df['winner_is_home_team'].values

In [17]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, shuffle=True)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=True) 

Before we create our model, we test the predictive performance of a simple Logistic Regression model first.

In [18]:
model_logistic_regression = models.Sequential([
    # Logistic Regression
    layers.InputLayer(input_shape=(26,)),
    layers.Dense(units=2, activation='sigmoid')
])

model_logistic_regression.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 2)                 54        
                                                                 
Total params: 54 (216.00 Byte)
Trainable params: 54 (216.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model_logistic_regression.compile(optimizer='adam', # optimizer type
              loss=tf.keras.losses.sparse_categorical_crossentropy, # loss function
              metrics=['accuracy'])
history_logistic_regression = model_logistic_regression.fit(x=X_train,y=y_train,
                     epochs=20,
                     validation_data=(X_val,y_val))

Epoch 1/20
28/28 [==============================] - 0s 4ms/step - loss: 111.6421 - accuracy: 0.4016 - val_loss: 98.4774 - val_accuracy: 0.4034
Epoch 2/20
28/28 [==============================] - 0s 2ms/step - loss: 87.0034 - accuracy: 0.4016 - val_loss: 74.0472 - val_accuracy: 0.4034
Epoch 3/20
28/28 [==============================] - 0s 3ms/step - loss: 62.6110 - accuracy: 0.4016 - val_loss: 49.0190 - val_accuracy: 0.4034
Epoch 4/20
28/28 [==============================] - 0s 3ms/step - loss: 37.8879 - accuracy: 0.4016 - val_loss: 24.6277 - val_accuracy: 0.4034
Epoch 5/20
28/28 [==============================] - 0s 2ms/step - loss: 13.6756 - accuracy: 0.3891 - val_loss: 2.9452 - val_accuracy: 0.4830
Epoch 6/20
28/28 [==============================] - 0s 2ms/step - loss: 3.1700 - accuracy: 0.5108 - val_loss: 2.7068 - val_accuracy: 0.5057
Epoch 7/20
28/28 [==============================] - 0s 2ms/step - loss: 2.8086 - accuracy: 0.4608 - val_loss: 2.7127 - val_accuracy: 0.4602
Epoch 8/20

Now we try our own Neural Network model.

In [62]:
model1 = models.Sequential([
    # NN
    layers.InputLayer(input_shape=(26,)),
    layers.Dense(units=128, use_bias=True), # Linear layer
    layers.Dense(units=64, use_bias=True, activation='relu'), # Non-Linear layer
    #layers.Flatten(),
    layers.Dense(units=64, activation='relu'), # An inner product of previous parameters with weights
    layers.Dense(units=1, activation='sigmoid')
    
])

model1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 128)               3456      
                                                                 
 dense_27 (Dense)            (None, 64)                8256      
                                                                 
 dense_28 (Dense)            (None, 64)                4160      
                                                                 
 dense_29 (Dense)            (None, 1)                 65        
                                                                 
Total params: 15937 (62.25 KB)
Trainable params: 15937 (62.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [65]:
model1.compile(optimizer='adam', # optimizer type
              loss=tf.keras.losses.binary_crossentropy, # loss function
              metrics=['accuracy'])

stop_here_please = EarlyStopping(patience=3)

history_1 = model1.fit(x=X_train,y=y_train,
                     epochs=20,
                     validation_data=(X_val,y_val),
                     )#callbacks=[stop_here_please]

Epoch 1/20
28/28 [==============================] - 1s 6ms/step - loss: 0.8303 - accuracy: 0.5222 - val_loss: 0.6665 - val_accuracy: 0.6307
Epoch 2/20
28/28 [==============================] - 0s 3ms/step - loss: 0.6831 - accuracy: 0.5768 - val_loss: 0.7297 - val_accuracy: 0.5966
Epoch 3/20
28/28 [==============================] - 0s 3ms/step - loss: 0.6878 - accuracy: 0.5813 - val_loss: 0.6648 - val_accuracy: 0.5966
Epoch 4/20
28/28 [==============================] - 0s 2ms/step - loss: 0.6787 - accuracy: 0.5643 - val_loss: 0.6626 - val_accuracy: 0.6023
Epoch 5/20
28/28 [==============================] - 0s 2ms/step - loss: 0.6783 - accuracy: 0.5882 - val_loss: 0.6742 - val_accuracy: 0.6136
Epoch 6/20
28/28 [==============================] - 0s 2ms/step - loss: 0.6737 - accuracy: 0.5836 - val_loss: 0.6530 - val_accuracy: 0.6250
Epoch 7/20
28/28 [==============================] - 0s 3ms/step - loss: 0.6633 - accuracy: 0.6075 - val_loss: 0.6637 - val_accuracy: 0.6250
Epoch 8/20
28/28 [==

In [66]:
model1.evaluate(x=X_test,y=y_test)

4/4 [==============================] - 0s 811us/step - loss: 0.6713 - accuracy: 0.6017


[0.6712800860404968, 0.6016949415206909]

In [67]:
lakers_reg_season_2000_stats = get_average_stats_over_period(conn, "lakers", nba_seasons[params_dict['Season']]['start'], nba_seasons[params_dict['Season']]['end'])
blazers_reg_season_2000_stats = get_average_stats_over_period(conn, "blazers", nba_seasons[params_dict['Season']]['start'], nba_seasons[params_dict['Season']]['end'])

In [68]:
lakers_reg_season_2000_stats = lakers_reg_season_2000_stats.drop('gamesCount', axis = 1)
blazers_reg_season_2000_stats = blazers_reg_season_2000_stats.drop('gamesCount', axis = 1)

In [69]:
lakers_reg_season_2000_stats = lakers_reg_season_2000_stats.add_prefix("hometeam_")
blazers_reg_season_2000_stats = blazers_reg_season_2000_stats.add_prefix("awayteam_")

In [70]:
lakers_vs_blazers_df = pd.concat([lakers_reg_season_2000_stats,blazers_reg_season_2000_stats], axis = 1)
lakers_vs_blazers_df

,hometeam_fieldGoalsMade,hometeam_fieldGoalsAttempted,hometeam_threePointersMade,hometeam_threePointersAttempted,hometeam_freeThrowsMade,hometeam_freeThrowsAttempted,hometeam_reboundsOffensive,hometeam_reboundsDefensive,hometeam_assists,hometeam_steals,...,awayteam_threePointersAttempted,awayteam_freeThrowsMade,awayteam_freeThrowsAttempted,awayteam_reboundsOffensive,awayteam_reboundsDefensive,awayteam_assists,awayteam_steals,awayteam_blocks,awayteam_foulsPersonal,awayteam_points
0,37.914634,81.52439,5.353659,15.54878,19.439024,28.45122,13.231707,31.5,23.02439,6.878049,...,12.890244,17.646341,23.158537,11.695122,30.256098,23.939024,8.195122,5.109756,21.280488,95.414634


In [71]:
model1.predict(lakers_vs_blazers_df)

1/1 [==============================] - 0s 27ms/step


array([[0.65145427]], dtype=float32)

In [ ]:
conn.close()